In [1]:
import torch
print(torch.__version__)
import torch.nn as nn

import torch.optim as optim
import torch.utils.data as data_utils

from torch.utils.data import DataLoader, Dataset, Sampler
from torch.utils.data.dataloader import default_collate
from torch.utils.tensorboard import SummaryWriter
from pytorch_lightning.metrics import Accuracy

import pandas as pd
import numpy as np

import torch.nn.functional as F

1.8.1+cu101


# TabNet

https://arxiv.org/pdf/1908.07442.pdf

## Global architecture

<img src="./images/global_arch.png" alt="Drawing" width="600px"/>

## Ghost Batch Normalization (GBN):

GBN позволяет нам обучать большие батчи и в то же время лучше обобщать. Суть GBN в том, что мы разделяем входной батч на суб-батчи равного размера (размером vbs - virtual batch size) и применяем к ним один и тот же слой BatchNorm. Все слои BatchNorm, используемые в модели, за исключением первого слоя BN, примененного к входным объектам, являются слоями GBN

In [2]:
class GBN(nn.Module):
    def __init__(self,inp,vbs=128,momentum=0.01):
        super().__init__()
        self.bn = nn.BatchNorm1d(inp,momentum=momentum)
        self.vbs = vbs
    def forward(self,x):
        chunk = torch.chunk(x,x.size(0)//self.vbs,0)
        res = [self.bn(y) for y in chunk]
        return torch.cat(res,0)

## Feature Transformer

<img src="./images/feature_transformer.png" alt="Drawing" width="600px"/>

__Feature Transformer__ - это место, где все выбранные фичи обрабатываются для генерации окончательного вывода. Каждый Feature Transformer состоит из нескольких Gated Linear Unit Blocks. GLU контролирует, какая информация должна быть разрешена для дальнейшего прохождения через сеть. Чтобы реализовать блок GLU, сначала мы __удваиваем__ размерность входных фичей в GLU, используя полносвязный слой(так как сама функция GLU делит размерность на 2). Затем мы нормализуем результирующую матрицу, используя слой GBN. Далее применяем __сигмоид ко второй половине__ полученных признаков и __умножаем результаты на первую половину поэлементно__ . Результат умножается на коэффициент масштабирования (в данном случае sqrt (0,5)) и добавляется ко входным данным следующего блока. Этот суммарный результат является вводом для следующего блока GLU в последовательности.<br><br>
Определенное число блоков GLU используется во всех decision step'ах (Shared across decision steps), чтобы повысить производительность и эффективность модели. Первый shared блок GLU (или первый decision step dependent блок, если нет shared блоков) уникален, поскольку он уменьшает размер входных функций до размера, равного n_a + n_d. n_a - это размерность фичей, вводимых в Attentive Transformer на следующем шаге, а n_d - размерность фичей, используемых для вычисления окончательных результатов. Эти объекты обрабатываются вместе, пока не достигнут разделителя. Активация ReLU применяется к вектору размерности n_d. Выходные данные всех decision step'ов суммируются и проходят через полносвязный слой, чтобы сопоставить их с нужными нам выходными размерностями.

## GLU Layer

Заметим, что можно объединить слои FC, GBN и GLU в один слой, назовем его просто GLU:

In [3]:
class GLU(nn.Module):
    def __init__(self,inp_dim,out_dim,fc=None,vbs=128):
        super().__init__()
        if fc:
            self.fc = fc
        else:
            self.fc = nn.Linear(inp_dim,out_dim*2)
        self.bn = GBN(out_dim*2,vbs=vbs) 
        
    def init_weights(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform(m.weight)
            # m.bias.data.fill_(0.001)
        
    def forward(self,input_data):
        output = self.fc(input_data)
        output = self.bn(output)
        output = torch.nn.functional.glu(output)
        
        return output

<img src="./images/glu.png" alt="Drawing" width="600px"/>

Реализуем наконец сам Feature Transformer

In [4]:
class FeatureTransformer(nn.Module):
    def __init__(self,inp_dim,out_dim,shared,n_ind,vbs=128):
        super().__init__()
        first = True
        self.shared = nn.ModuleList()
        if shared:
            self.shared.append(GLU(inp_dim,out_dim,shared[0],vbs=vbs))
            first= False    
            for fc in shared[1:]:
                self.shared.append(GLU(out_dim,out_dim,fc,vbs=vbs))
        else:
            self.shared = None
        self.independ = nn.ModuleList()
        if first:
            self.independ.append(GLU(inp,out_dim,vbs=vbs))
        for x in range(first, n_ind):
            self.independ.append(GLU(out_dim,out_dim,vbs=vbs))
        self.scale = torch.sqrt(torch.tensor([.5]))
    def forward(self,x):
        if self.shared:
            x = self.shared[0](x)
            for glu in self.shared[1:]:
                x = torch.add(x, glu(x))
                x = x*self.scale
        for glu in self.independ:
            x = torch.add(x, glu(x))
            x = x*self.scale
        return x

## Attentive Transformer

<img src="./images/attentive_transformer.png" alt="Drawing" width="300px"/>

Здесь модель учит взаимосвязи между фичами и решает, какие фичи передать Feature Transformer на текущем decision step'е. Каждый Attentive Transformer состоит из полносвязного слоя, слоя GBN и Sparsemax. Attentive Transformer на каждом decision step'е получает входные фичи, обработанные фичи из предыдущего шага и Prior scales используемых фич. Prior scales представлена матрицей размера batch_size x input_features. Она инициализируется единицами и передается в Attentive Transformer каждого decision step'а и обновляется. Также есть параметр релаксации, который ограничивает то, сколько раз можно использовать определенную фичу в forward pass. Более высокое значение означает, что модель может повторно использовать одну и ту же фичу несколько раз.

In [5]:
class AttentionTransformer(nn.Module):
    def __init__(self,d_a,inp_dim,relax,vbs=128):
        super().__init__()
        self.fc = nn.Linear(d_a,inp_dim)
        self.bn = GBN(inp_dim,vbs=vbs)
        self.smax = Sparsemax()
        self.r = relax
    #a:feature from previous decision step
    def forward(self,a,priors): 
        a = self.bn(self.fc(a)) 
        mask = self.smax(a*priors) 
        priors =priors*(self.r-mask)  #updating the prior
        return mask

### Sparsemax https://github.com/KrisKorrel/sparsemax-pytorch/blob/master/sparsemax.py

__Sparsemax__ - это функция активации, как и softmax, но, как следует из названия, распределение более разреженное. То есть, по сравнению с softmax, некоторые числа в распределении вероятности выхода намного ближе к 1, а другие - к 0. Это позволяет модели более эффективно выбирать релевантные фичи на каждом decision step'е. Мы будем использовать sparsemax, чтобы спроецировать маску для feature selection на каждом шаге

Чтобы еще больше увеличить разреженность маски, можно добавить метод регуляризации разреженности, чтобы "наказывать" менее разреженные маски. Это может быть реализовано на каждом шаге принятия решения следующим образом:

(mask*torch.log(mask+1e-10)).mean() #F(x)= -∑xlog(x+eps)

Сумма этого значения по всем шагам решения может быть добавлена к общему лоссу (после умножения на константу регуляризации λ)

In [6]:
"""
Sparsemax activation function.
Pytorch implementation of Sparsemax function from:
-- "From Softmax to Sparsemax: A Sparse Model of Attention and Multi-Label Classification"
-- André F. T. Martins, Ramón Fernandez Astudillo (http://arxiv.org/abs/1602.02068)
"""

from __future__ import division

import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Sparsemax(nn.Module):
    """Sparsemax function."""

    def __init__(self, dim=None):
        """Initialize sparsemax activation
        
        Args:
            dim (int, optional): The dimension over which to apply the sparsemax function.
        """
        super(Sparsemax, self).__init__()

        self.dim = -1 if dim is None else dim

    def forward(self, input):
        """Forward function.
        Args:
            input (torch.Tensor): Input tensor. First dimension should be the batch size
        Returns:
            torch.Tensor: [batch_size x number_of_logits] Output tensor
        """
        # Sparsemax currently only handles 2-dim tensors,
        # so we reshape to a convenient shape and reshape back after sparsemax
        input = input.transpose(0, self.dim)
        original_size = input.size()
        input = input.reshape(input.size(0), -1)
        input = input.transpose(0, 1)
        dim = 1

        number_of_logits = input.size(dim)

        # Translate input by max for numerical stability
        input = input - torch.max(input, dim=dim, keepdim=True)[0].expand_as(input)

        # Sort input in descending order.
        # (NOTE: Can be replaced with linear time selection method described here:
        # http://stanford.edu/~jduchi/projects/DuchiShSiCh08.html)
        zs = torch.sort(input=input, dim=dim, descending=True)[0]
        range = torch.arange(start=1, end=number_of_logits + 1, step=1, dtype=input.dtype).view(1, -1)
        range = range.expand_as(zs)

        # Determine sparsity of projection
        bound = 1 + range * zs
        cumulative_sum_zs = torch.cumsum(zs, dim)
        is_gt = torch.gt(bound, cumulative_sum_zs).type(input.type())
        k = torch.max(is_gt * range, dim, keepdim=True)[0]

        # Compute threshold function
        zs_sparse = is_gt * zs

        # Compute taus
        taus = (torch.sum(zs_sparse, dim, keepdim=True) - 1) / k
        taus = taus.expand_as(input)

        # Sparsemax
        self.output = torch.max(torch.zeros_like(input), input - taus)

        # Reshape back to original shape
        output = self.output
        output = output.transpose(0, 1)
        output = output.reshape(original_size)
        output = output.transpose(0, self.dim)

        return output

    def backward(self, grad_output):
        """Backward function."""
        dim = 1

        nonzeros = torch.ne(self.output, 0)
        sum = torch.sum(grad_output * nonzeros, dim=dim) / torch.sum(nonzeros, dim=dim)
        self.grad_input = nonzeros * (grad_output - sum.expand_as(grad_output))

        return self.grad_input

## Собираем все слои в одну большую сеть

Объединим Attention Transformer и Feature Transformer в Decision Step:

In [7]:
class DecisionStep(nn.Module):
    def __init__(self,inp_dim,n_d,n_a,shared,n_ind,relax,vbs=128):
        super().__init__()
        self.fea_tran = FeatureTransformer(inp_dim,n_d+n_a,shared,n_ind,vbs)
        self.atten_tran =  AttentionTransformer(n_a,inp_dim,relax,vbs)
    def forward(self,x,a,priors):
        mask = self.atten_tran(a,priors)
        sparse_loss = ((-1)*mask*torch.log(mask+1e-10)).mean()
        x = self.fea_tran(x*mask)
        return x,sparse_loss

Наконец, мы можем завершить модель, объединив несколько Decision step'ов вместе

In [8]:
class TabNet(nn.Module):
    def __init__(self,inp_dim,final_out_dim,n_d=64,n_a=64,
n_shared=2,n_ind=2,n_steps=5,relax=1.2,vbs=128):
        super().__init__()
        if n_shared>0:
            self.shared = nn.ModuleList()
            self.shared.append(nn.Linear(inp_dim,2*(n_d+n_a)))
            for x in range(n_shared-1):
                self.shared.append(nn.Linear(n_d+n_a,2*(n_d+n_a)))
        else:
            self.shared=None
        self.first_step = FeatureTransformer(inp_dim,n_d+n_a,self.shared,n_ind) 
        self.steps = nn.ModuleList()
        for x in range(n_steps-1):
            self.steps.append(DecisionStep(inp_dim,n_d,n_a,self.shared,n_ind,relax,vbs))
        self.fc = nn.Linear(n_d,final_out_dim)
        self.bn = nn.BatchNorm1d(inp_dim)
        self.n_d = n_d
    def forward(self,x):
        x = self.bn(x)
        x_a = self.first_step(x)[:,self.n_d:]
        sparse_loss = torch.zeros(1)
        out = torch.zeros(x.size(0),self.n_d)
        priors = torch.ones(x.shape)
        for step in self.steps:
            x_te,l = step(x,x_a,priors)
            out += F.relu(x_te[:,:self.n_d])
            x_a = x_te[:,self.n_d:]
            sparse_loss += l
        return self.fc(out)

## Обучаем TabNet на датасете adult

In [9]:
EPOCHS = 500
EMBEDDING_SIZE = 5
BATCH_SIZE = 512
INPUT_SIZE = 14
NROF_OUT_CLASSES = 1
LEARNING_RATE = 3e-4
TRAIN_PATH = '../data/train_adult.pickle'
VALID_PATH = '../data/valid_adult.pickle'

In [83]:
def run_train(model, train_loader, test_loader):
    step = 0
    for epoch in range(EPOCHS):
        model.train()

        for features, label in train_loader:
            # Reset gradients
            optimizer.zero_grad()

            output = model(features)
            # Calculate error and backpropagate
            loss = criterion(output, torch.unsqueeze(label, 1))
            loss.backward()
            acc = accuracy(torch.sigmoid(output), torch.unsqueeze(label.long(), 1)).item()

            # Update weights with gradients
            optimizer.step()
            
            train_writer.add_scalar('CrossEntropyLoss', loss, step)
            train_writer.add_scalar('Accuracy', acc, step)

            step += 1

            if step % 100 == 0:
                print('EPOCH %d STEP %d : train_loss: %f train_acc: %f' %
                      (epoch, step, loss.item(), acc))
        
#         train_writer.add_histogram('hidden_layer', model.linear1.weight.data, step)

        
        # Run validation
        running_loss = []
        valid_scores = []
        valid_labels = []
        model.eval()
        with torch.no_grad():
            for features, label in test_loader:
                output = model(features)
                # Calculate error and backpropagate
                loss = criterion(output, torch.unsqueeze(label, 1))

                running_loss.append(loss.item())
                valid_scores.extend((torch.sigmoid(output)>0.5).long())
                valid_labels.extend(torch.unsqueeze(label, 1))

        valid_accuracy = accuracy(torch.tensor(valid_scores), torch.tensor(valid_labels).long()).item()

        valid_writer.add_scalar('CrossEntropyLoss', np.mean(running_loss), step)
        valid_writer.add_scalar('Accuracy', valid_accuracy, step)

        print('EPOCH %d : valid_loss: %f valid_acc: %f' % (epoch, np.mean(running_loss), valid_accuracy))
        
    return step

In [84]:
# Функция создания train и test даталоадера
def create_data_loader(train_dataset, train_sampler,
                       test_dataset, test_sampler):
    train_loader = DataLoader(dataset=train_dataset, sampler=train_sampler,
                              batch_size=BATCH_SIZE, collate_fn=default_collate,
                              shuffle=False)

    test_loader = DataLoader(dataset=test_dataset, sampler=test_sampler,
                             batch_size=BATCH_SIZE, collate_fn=default_collate,
                             shuffle=False)

    return train_loader, test_loader

In [85]:
import pickle
import torch

from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, dataset_path):
        super().__init__()
        with open(dataset_path, 'rb') as f:
            data, self.nrof_emb_categories, self.unique_categories = pickle.load(f)

        self.embedding_columns = ['workclass_cat', 'education_cat', 'marital-status_cat', 'occupation_cat',
                                  'relationship_cat', 'race_cat',
                                  'sex_cat', 'native-country_cat']
        self.nrof_emb_categories = {key + '_cat': val for key, val in self.nrof_emb_categories.items()}
        self.numeric_columns = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
                                'hours-per-week']

        self.columns = self.embedding_columns + self.numeric_columns

        self.X = data[self.columns].reset_index(drop=True)
        self.y = np.asarray([0 if el == '<50k' else 1 for el in data['salary'].values], dtype=np.int32)

        return

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):

#         row = self.X.take([idx], axis=0)

#         row = {col: torch.tensor(row[col].values, dtype=torch.float32) for i, col in enumerate(self.columns)}
        

        return np.float32(self.X.loc[idx,:]), np.float32(self.y[idx])

In [86]:
class CustomSampler(Sampler):

    # Конструктор, где инициализируем индексы элементов
    def __init__(self, data):
        self.data_indices = np.arange(len(data))

        shuffled_indices = np.random.permutation(len(self.data_indices))

        self.data_indices = np.ascontiguousarray(self.data_indices)[shuffled_indices]

        return

    def __len__(self):
        return len(self.data_indices)

    # Возращает итератор,
    # который будет возвращать индексы из перемешанного датасета
    def __iter__(self):
        return iter(self.data_indices)

In [87]:
# Создание train даталоадера и test даталоадера
train_ds = CustomDataset(TRAIN_PATH)
train_sampler = CustomSampler(train_ds.X)

test_ds = CustomDataset(VALID_PATH)
test_sampler = CustomSampler(test_ds.X)

train_loader, test_loader = create_data_loader(train_ds, train_sampler,
                                               test_ds, test_sampler)

In [88]:
train_writer = SummaryWriter('./logs/train')
valid_writer = SummaryWriter('./logs/valid')

In [89]:
model = TabNet(INPUT_SIZE, NROF_OUT_CLASSES)


criterion = nn.BCEWithLogitsLoss()
accuracy = Accuracy()

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [90]:
step = run_train(model, train_loader, test_loader)

EPOCH 0 : valid_loss: 0.567650 valid_acc: 0.762629
EPOCH 1 STEP 100 : train_loss: 0.375814 train_acc: 0.832031
EPOCH 1 : valid_loss: 0.409507 valid_acc: 0.828036
EPOCH 2 : valid_loss: 0.360505 valid_acc: 0.837709
EPOCH 3 STEP 200 : train_loss: 0.343188 train_acc: 0.822266
EPOCH 3 : valid_loss: 0.359711 valid_acc: 0.835099
EPOCH 4 : valid_loss: 0.355420 valid_acc: 0.837402
EPOCH 5 STEP 300 : train_loss: 0.362053 train_acc: 0.824219
EPOCH 5 : valid_loss: 0.347919 valid_acc: 0.841241
EPOCH 6 : valid_loss: 0.348722 valid_acc: 0.842008
EPOCH 7 STEP 400 : train_loss: 0.345664 train_acc: 0.841797
EPOCH 7 : valid_loss: 0.342446 valid_acc: 0.840319
EPOCH 8 : valid_loss: 0.336144 valid_acc: 0.845386
EPOCH 9 STEP 500 : train_loss: 0.349295 train_acc: 0.855469
EPOCH 9 : valid_loss: 0.332235 valid_acc: 0.846307
EPOCH 10 : valid_loss: 0.332584 valid_acc: 0.843237
EPOCH 11 STEP 600 : train_loss: 0.356749 train_acc: 0.833984
EPOCH 11 : valid_loss: 0.333741 valid_acc: 0.844772
EPOCH 12 : valid_loss: 0.

KeyboardInterrupt: 